# 2012-2015년 데이터로 모델 만들기(남자)

In [1]:
import pandas as pd
from itertools import *

In [2]:
dataset_male = pd.read_csv('./hospital_male_0802_test2.csv', encoding='cp949')
dataset_male = dataset_male[dataset_male['yyyy'].astype(str).str.contains('2012|2013|2014|2015')].reset_index(drop=True)#2016년 전만 추출
male_df = dataset_male[['area','mm','weekday','heat_wave','cold_wave','total_pop','elderly_ratio',
                          'pm10_7b','diff_hum','diff_temp','avg_ps',
                          'o3','avg_age','min_max_ps', 'avg_hum','frequency']]
# fixed_df = dataset2[['area', 'mm', 'frequency','total_pop', 'elderly_ratio','so2','pm10','rhm_min_avg_dif','min_max_Ta','avg_ps']]
male_df

,area,mm,weekday,heat_wave,cold_wave,total_pop,elderly_ratio,pm10_7b,diff_hum,diff_temp,avg_ps,o3,avg_age,min_max_ps,avg_hum,frequency
0,강원,1,6,0,0,772718,12.460044,30.158333,65.0,12.3,1023.886667,0.027750,39.4,6.340000,58.333333,3.0
1,경기,1,6,0,0,6020446,7.432174,28.537404,27.0,7.9,1024.875000,0.023165,36.1,5.500000,50.733333,4.0
2,경남,1,6,0,0,1665308,9.179443,26.719907,47.0,9.9,1022.950000,0.027277,37.4,5.200000,44.916667,2.0
3,경북,1,6,0,0,1354649,12.336406,25.336806,70.0,12.0,1023.268000,0.027274,39.5,5.688000,52.377778,6.0
4,광주,1,6,0,0,726529,7.679253,18.958333,15.0,3.9,1025.250000,0.027300,35.3,4.100000,75.833333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,전남,12,3,0,0,953881,16.198981,46.708333,65.0,12.3,1031.053846,0.035985,42.1,6.892308,66.764033,0.0
24833,전북,12,3,0,0,930255,14.539347,72.548882,50.0,10.0,1031.300000,0.028738,40.9,7.014286,73.750000,1.0
24834,제주,12,3,0,0,313428,11.063466,56.156250,33.0,4.9,1030.792308,0.044719,38.7,7.523077,59.915541,0.0
24835,충남,12,3,0,0,1054439,13.477024,79.543252,55.0,9.0,1031.513333,0.024484,40.2,6.800000,74.725225,0.0


In [3]:
male_data = male_df.sample(frac=0.9, random_state=786) 
male_data_unseen = male_df.drop(male_data.index) 

male_data.reset_index(drop=True, inplace=True) 
male_data_unseen.reset_index(drop=True, inplace=True) 

print('Data for Modeling: ' + str(male_data.shape)) 
print('Unseen Data For Predictions: ' + str(male_data_unseen.shape)) 


Data for Modeling: (22353, 16)
Unseen Data For Predictions: (2484, 16)


## 모델 환경 설정 (Settinf up Environment in PyCaret)

- pycaret을 사용하기 전에 pycaret에 맞게 데이터를 설정
- set_up() 함수를 사용
- 기본적으로 data와 target을 입력

In [4]:
# 자동으로 데이터 유형 지정 
from pycaret.regression import *
s = setup(male_data, target = 'frequency', session_id=123, categorical_features=['area','mm','weekday','heat_wave',
                                                                                'cold_wave'])

,Description,Value
0,Session id,123
1,Target,frequency
2,Target type,Regression
3,Data shape,"(22353, 20)"
4,Train data shape,"(15647, 20)"
5,Test data shape,"(6706, 20)"
6,Numeric features,14
7,Categorical features,5
8,Preprocess,True
9,Imputation type,simple


#### set_up(): pycaret을 사용하기 위한 data setting

- session_id: random_state와 같은 개념으로 같은 결과가 나올 수 있게 seed를 고정
- data: train 데이터 입력
- target = target 변수 이름을 입력

## 모델 생성 - 비교

- 여러 모델을 적합하여 성능을 비교하는 단계
- compare_models(): 다양한 모델 적합 후 성능 비교
    - fold: cross_validation의 fold를 지정 (default = 10)
    - sort: 정렬기준 지표 설정
    - n_select: 상위 n개의 모델 결과만 출력

In [5]:
# et_male = create_model('et')
male_best_model_5 = compare_models(sort='RMSE', n_select = 5)
male_tuned_top5 = [tune_model(i, optimize = 'RMSE', n_iter=10) for i in male_best_model_5]
model_male = blend_models(estimator_list=male_tuned_top5, optimize = 'RMSE')

# final_model = finalize_model(model_male)
# prediction = predict_model(final_model, data=male_data_unseen)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,0.9132,1.5551,1.2467,0.4567,0.4898,0.4495,0.0300
lr,Linear Regression,0.9134,1.5555,1.2468,0.4566,0.4901,0.4504,0.1700
ridge,Ridge Regression,0.9133,1.5554,1.2468,0.4566,0.4900,0.4503,0.0290
lar,Least Angle Regression,0.9134,1.5555,1.2468,0.4566,0.4901,0.4504,0.0270
omp,Orthogonal Matching Pursuit,0.9146,1.5922,1.2615,0.4438,0.4924,0.4395,0.0350
et,Extra Trees Regressor,0.9432,1.6191,1.2722,0.4343,0.5033,0.4436,0.1570
en,Elastic Net,0.9588,1.6654,1.2902,0.4181,0.5084,0.4126,0.0300
lasso,Lasso Regression,0.9595,1.6686,1.2914,0.4170,0.5091,0.4102,0.0300
lightgbm,Light Gradient Boosting Machine,0.9733,1.6809,1.2959,0.4127,0.5286,0.4413,0.0450
huber,Huber Regressor,0.9508,1.6877,1.2988,0.4104,0.4980,0.4275,0.0340


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9219,1.6375,1.2797,0.4176,0.4924,0.4654
1,0.9297,1.6373,1.2796,0.4204,0.4933,0.4549
2,0.8931,1.4753,1.2146,0.4815,0.4790,0.4291
3,0.9051,1.5751,1.2550,0.4580,0.4847,0.4515
4,0.9109,1.4841,1.2182,0.4622,0.4973,0.4395
5,0.8911,1.4759,1.2149,0.5101,0.4774,0.4243
6,0.9127,1.5369,1.2397,0.4679,0.4933,0.4452
7,0.9100,1.4904,1.2208,0.4609,0.4906,0.4784
8,0.9572,1.7009,1.3042,0.4264,0.5072,0.4803


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


...

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9222,1.6393,1.2804,0.4170,0.4928,0.4655
1,0.9283,1.6340,1.2783,0.4215,0.4927,0.4539
2,0.8909,1.4685,1.2118,0.4839,0.4783,0.4291
3,0.9037,1.5724,1.2540,0.4589,0.4844,0.4505
4,0.9099,1.4807,1.2168,0.4635,0.4975,0.4391
5,0.8917,1.4787,1.2160,0.5092,0.4780,0.4237
6,0.9121,1.5346,1.2388,0.4686,0.4933,0.4446
7,0.9095,1.4888,1.2202,0.4615,0.4905,0.4788
8,0.9572,1.7023,1.3047,0.4259,0.5072,0.4801


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9217,1.6375,1.2796,0.4176,0.4924,0.4657
1,0.9293,1.6371,1.2795,0.4204,0.4933,0.4546
2,0.8926,1.4735,1.2139,0.4822,0.4789,0.4296
3,0.9047,1.5739,1.2545,0.4584,0.4846,0.4517
4,0.9104,1.4826,1.2176,0.4628,0.4972,0.4396
5,0.8914,1.4766,1.2152,0.5099,0.4776,0.4246
6,0.9129,1.5371,1.2398,0.4678,0.4935,0.4457
7,0.9102,1.4900,1.2207,0.4610,0.4907,0.4789
8,0.9570,1.7007,1.3041,0.4265,0.5072,0.4805


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

# 2016년 예측(남자)

In [6]:
test_male = pd.read_csv('./hospital_male_0802_test2.csv', encoding='cp949') #이 csv 파일은 2016년 데이터까지 포함한 것
# dataset3.drop(columns=['Unnamed: 0'], inplace=True)
test_male = test_male[['area','yyyy','dd','mm','weekday','heat_wave','cold_wave','total_pop','elderly_ratio',
          'pm10_7b','diff_hum','diff_temp','avg_ps',
          'o3','avg_age','min_max_ps', 'avg_hum','frequency']]
test_male

,area,yyyy,dd,mm,weekday,heat_wave,cold_wave,total_pop,elderly_ratio,pm10_7b,diff_hum,diff_temp,avg_ps,o3,avg_age,min_max_ps,avg_hum,frequency
0,강원,2011,1,12,3,0,0,773278,12.405500,NaN,33.314706,3.845070,1027.575000,0.027986,39.3,2.993750,83.354839,NaN
1,경기,2011,1,12,3,0,0,6015357,7.388123,NaN,28.227586,5.413208,1025.875000,0.024401,36.0,3.175000,65.448276,NaN
2,경남,2011,1,12,3,0,0,1664755,9.147472,NaN,19.566667,3.112821,1023.921429,0.027836,37.3,2.578571,74.583333,NaN
3,경북,2011,1,12,3,0,0,1355727,12.284700,NaN,28.800000,3.338182,1026.137500,0.027465,39.5,2.684000,81.000000,NaN
4,광주,2011,1,12,3,0,0,725606,7.629485,NaN,23.066667,3.840000,1022.900000,0.021888,35.3,3.000000,89.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31581,전남,2016,31,12,5,0,0,951889,16.677050,33.635000,49.000000,13.700000,1030.607692,0.025507,42.6,2.573333,65.555094,NaN
31582,전북,2016,31,12,5,0,0,927505,15.011563,52.149758,42.000000,14.500000,1030.586667,0.021604,41.4,3.466667,72.161486,NaN
31583,제주,2016,31,12,5,0,0,322545,11.278426,28.460474,31.000000,8.000000,1029.900000,0.039094,39.2,3.046667,60.569257,NaN
31584,충남,2016,31,12,5,0,0,1064765,13.768860,71.473958,41.000000,11.700000,1030.100000,0.019109,40.7,2.520000,75.118243,NaN


In [7]:
test_male2 = test_male[test_male['yyyy'].astype(str).str.contains('2016')].reset_index(drop=True)#2016년만 추출
test_male2

,area,yyyy,dd,mm,weekday,heat_wave,cold_wave,total_pop,elderly_ratio,pm10_7b,diff_hum,diff_temp,avg_ps,o3,avg_age,min_max_ps,avg_hum,frequency
0,강원,2016,1,1,4,0,0,781185,14.022671,61.954167,73.0,18.9,1030.230000,0.024237,41.4,8.010000,58.196911,NaN
1,경기,2016,1,1,4,0,0,6306634,8.966558,42.487621,45.0,12.1,1031.633333,0.014370,38.0,7.016667,70.844595,NaN
2,경남,2016,1,1,4,0,0,1695187,10.962802,29.128224,79.0,17.7,1031.175000,0.029582,39.3,5.616667,52.099421,NaN
3,경북,2016,1,1,4,0,0,1356568,14.384461,38.389385,66.0,20.0,1030.309524,0.033232,41.5,6.233333,57.314554,NaN
4,광주,2016,1,1,4,0,0,729995,9.572120,20.475000,40.0,10.0,1032.200000,0.025175,37.4,5.850000,62.837838,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,전남,2016,31,12,5,0,0,951889,16.677050,33.635000,49.0,13.7,1030.607692,0.025507,42.6,2.573333,65.555094,NaN
6218,전북,2016,31,12,5,0,0,927505,15.011563,52.149758,42.0,14.5,1030.586667,0.021604,41.4,3.466667,72.161486,NaN
6219,제주,2016,31,12,5,0,0,322545,11.278426,28.460474,31.0,8.0,1029.900000,0.039094,39.2,3.046667,60.569257,NaN
6220,충남,2016,31,12,5,0,0,1064765,13.768860,71.473958,41.0,11.7,1030.100000,0.019109,40.7,2.520000,75.118243,NaN


In [8]:
test_male3 = test_male2.drop(columns=['frequency','yyyy','dd'])

In [9]:
male_final_model = finalize_model(model_male)
prediction_male = predict_model(male_final_model, data = test_male3)

In [10]:
# prediction_male['Label'] = prediction_male['Label'].round(0)
# prediction_male[['Label']]

# 2012-2015년 데이터로 모델 만들기(여자)

In [11]:
dataset_female = pd.read_csv('./hospital_female_0802_test2.csv', encoding='cp949')
dataset_female = dataset_female[dataset_female['yyyy'].astype(str).str.contains('2012|2013|2014|2015')].reset_index(drop=True)#2016년 전만 추출
female_df = dataset_female[['area','mm','weekday','heat_wave','cold_wave','total_pop','elderly_ratio',
          'pm10_7b','diff_hum','diff_temp','avg_ps',
          'avg_age','min_max_ps', 'sum_rn','frequency']]
# fixed_df = dataset2[['area', 'mm', 'frequency','total_pop', 'elderly_ratio','so2','pm10','rhm_min_avg_dif','min_max_Ta','avg_ps']]
female_df

,area,mm,weekday,heat_wave,cold_wave,total_pop,elderly_ratio,pm10_7b,diff_hum,diff_temp,avg_ps,avg_age,min_max_ps,sum_rn,frequency
0,강원,1,6,0,0,762617,18.053361,30.158333,65.0,12.3,1023.886667,42.4,6.340000,0.173611,3.0
1,경기,1,6,0,0,5928150,10.649039,28.537404,27.0,7.9,1024.875000,37.7,5.500000,0.037736,5.0
2,경남,1,6,0,0,1644251,15.076105,26.719907,47.0,9.9,1022.950000,40.5,5.200000,0.065789,1.0
3,경북,1,6,0,0,1342455,19.248690,25.336806,70.0,12.0,1023.268000,43.2,5.688000,0.721053,1.0
4,광주,1,6,0,0,738784,11.037868,18.958333,15.0,3.9,1025.250000,37.6,4.100000,0.700000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,전남,12,3,0,0,955115,24.847060,46.708333,65.0,12.3,1031.053846,45.9,6.892308,1.608108,3.0
24833,전북,12,3,0,0,939456,21.104873,72.548882,50.0,10.0,1031.300000,44.2,7.014286,0.967742,0.0
24834,제주,12,3,0,0,310967,16.470236,56.156250,33.0,4.9,1030.792308,41.5,7.523077,0.971429,0.0
24835,충남,12,3,0,0,1023210,19.459055,79.543252,55.0,9.0,1031.513333,43.1,6.800000,0.196970,0.0


In [12]:
female_data = female_df.sample(frac=0.9, random_state=786) 
female_data_unseen = female_df.drop(female_data.index) 

female_data.reset_index(drop=True, inplace=True) 
female_data_unseen.reset_index(drop=True, inplace=True) 

print('Data for Modeling: ' + str(female_data.shape)) 
print('Unseen Data For Predictions: ' + str(female_data_unseen.shape)) 



Data for Modeling: (22353, 15)
Unseen Data For Predictions: (2484, 15)


In [13]:
# 자동으로 데이터 유형 지정 
from pycaret.regression import *
s = setup(female_data, target = 'frequency', session_id=123, categorical_features=['area','mm','weekday','heat_wave',
                                                                                'cold_wave'])

,Description,Value
0,Session id,123
1,Target,frequency
2,Target type,Regression
3,Data shape,"(22353, 19)"
4,Train data shape,"(15647, 19)"
5,Test data shape,"(6706, 19)"
6,Numeric features,13
7,Categorical features,5
8,Preprocess,True
9,Imputation type,simple


In [14]:
# et_female = create_model('et')
female_best_model_5 = compare_models(sort='RMSE', n_select = 5)
# tuned_top5 = [tune_model(i, optimize = 'RMSE', n_iter=10) for i in best_model_5]
# model_female = stack_models(tuned_top5)
model_female = blend_models(female_best_model_5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.9028,1.5014,1.2251,0.4499,0.4841,0.4581,0.0310
ridge,Ridge Regression,0.9028,1.5013,1.2251,0.4499,0.4841,0.4580,0.0300
lar,Least Angle Regression,0.9028,1.5014,1.2251,0.4499,0.4841,0.4581,0.0300
br,Bayesian Ridge,0.9026,1.5013,1.2251,0.4500,0.4840,0.4574,0.0300
omp,Orthogonal Matching Pursuit,0.9017,1.5443,1.2426,0.4342,0.4862,0.4452,0.0340
et,Extra Trees Regressor,0.9078,1.5469,1.2436,0.4333,0.4873,0.4388,0.1450
en,Elastic Net,0.9552,1.6278,1.2757,0.4035,0.5033,0.4284,0.0310
lightgbm,Light Gradient Boosting Machine,0.9468,1.6319,1.2771,0.4022,0.5146,0.4221,0.0450
lasso,Lasso Regression,0.9597,1.6638,1.2896,0.3904,0.5105,0.4130,0.0300
gbr,Gradient Boosting Regressor,0.9924,1.6772,1.2948,0.3858,0.5325,0.4260,0.2880


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.8941,1.4383,1.1993,0.4585,0.4764,0.4608
1,0.9208,1.5324,1.2379,0.4522,0.4921,0.4498
2,0.9044,1.5281,1.2362,0.4563,0.4848,0.4441
3,0.8692,1.4679,1.2116,0.4563,0.4671,0.4432
4,0.8979,1.5509,1.2453,0.4774,0.4763,0.4132
5,0.9006,1.4740,1.2141,0.4265,0.4879,0.4777
6,0.9330,1.6232,1.2740,0.4153,0.5003,0.4497
7,0.8725,1.4374,1.1989,0.4617,0.4727,0.4636
8,0.9006,1.4674,1.2114,0.4582,0.4859,0.4664


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

# 2016년 예측(여자)

In [15]:
test_female = pd.read_csv('./hospital_female_0802_test2.csv', encoding='cp949') #이 csv 파일은 2016년 데이터까지 포함한 것
# dataset3.drop(columns=['Unnamed: 0'], inplace=True)
test_female = test_female[['area','yyyy','dd','mm','weekday','heat_wave','cold_wave','total_pop','elderly_ratio',
          'pm10_7b','diff_hum','diff_temp','avg_ps',
          'avg_age','min_max_ps', 'sum_rn','frequency']]
test_female

,area,yyyy,dd,mm,weekday,heat_wave,cold_wave,total_pop,elderly_ratio,pm10_7b,diff_hum,diff_temp,avg_ps,avg_age,min_max_ps,sum_rn,frequency
0,강원,2011,1,12,3,0,0,763170,17.989439,NaN,33.314706,3.845070,1027.575000,42.3,2.993750,3.038889,NaN
1,경기,2011,1,12,3,0,0,5922058,10.605097,NaN,28.227586,5.413208,1025.875000,37.7,3.175000,0.009434,NaN
2,경남,2011,1,12,3,0,0,1644010,15.039264,NaN,19.566667,3.112821,1023.921429,40.5,2.578571,1.789474,NaN
3,경북,2011,1,12,3,0,0,1343468,19.192046,NaN,28.800000,3.338182,1026.137500,43.1,2.684000,2.185965,NaN
4,광주,2011,1,12,3,0,0,737858,10.984634,NaN,23.066667,3.840000,1022.900000,37.6,3.000000,3.200000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31581,전남,2016,31,12,5,0,0,952025,25.227174,33.635000,49.000000,13.700000,1030.607692,46.4,2.573333,0.000000,NaN
31582,전북,2016,31,12,5,0,0,937286,21.548385,52.149758,42.000000,14.500000,1030.586667,44.7,3.466667,0.000000,NaN
31583,제주,2016,31,12,5,0,0,319052,16.552474,28.460474,31.000000,8.000000,1029.900000,41.9,3.046667,0.000000,NaN
31584,충남,2016,31,12,5,0,0,1031962,19.719912,71.473958,41.000000,11.700000,1030.100000,43.5,2.520000,0.045455,NaN


In [16]:
test_female2 = test_female[test_female['yyyy'].astype(str).str.contains('2016')].reset_index(drop=True)#2016년만 추출
test_female2

,area,yyyy,dd,mm,weekday,heat_wave,cold_wave,total_pop,elderly_ratio,pm10_7b,diff_hum,diff_temp,avg_ps,avg_age,min_max_ps,sum_rn,frequency
0,강원,2016,1,1,4,0,0,768008,19.916980,61.954167,73.0,18.9,1030.230000,44.5,8.010000,0.000000,NaN
1,경기,2016,1,1,4,0,0,6229840,12.194759,42.487621,45.0,12.1,1031.633333,39.7,7.016667,0.000000,NaN
2,경남,2016,1,1,4,0,0,1669577,16.753405,29.128224,79.0,17.7,1031.175000,42.4,5.616667,0.000000,NaN
3,경북,2016,1,1,4,0,0,1344592,21.253882,38.389385,66.0,20.0,1030.309524,45.1,6.233333,0.000000,NaN
4,광주,2016,1,1,4,0,0,742807,13.091826,20.475000,40.0,10.0,1032.200000,39.7,5.850000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,전남,2016,31,12,5,0,0,952025,25.227174,33.635000,49.0,13.7,1030.607692,46.4,2.573333,0.000000,NaN
6218,전북,2016,31,12,5,0,0,937286,21.548385,52.149758,42.0,14.5,1030.586667,44.7,3.466667,0.000000,NaN
6219,제주,2016,31,12,5,0,0,319052,16.552474,28.460474,31.0,8.0,1029.900000,41.9,3.046667,0.000000,NaN
6220,충남,2016,31,12,5,0,0,1031962,19.719912,71.473958,41.0,11.7,1030.100000,43.5,2.520000,0.045455,NaN


In [17]:
test_female3 = test_female2.drop(columns=['frequency','yyyy','dd'])

In [18]:
female_final_model = finalize_model(model_female)
prediction_female = predict_model(female_final_model, data = test_female3)

In [19]:
# prediction_female['Label'] = prediction_female['Label'].round(0)
# prediction_female[['Label']]

# 남자 여자 예측 데이터 합치기

In [20]:
test_male2

,area,yyyy,dd,mm,weekday,heat_wave,cold_wave,total_pop,elderly_ratio,pm10_7b,diff_hum,diff_temp,avg_ps,o3,avg_age,min_max_ps,avg_hum,frequency
0,강원,2016,1,1,4,0,0,781185,14.022671,61.954167,73.0,18.9,1030.230000,0.024237,41.4,8.010000,58.196911,NaN
1,경기,2016,1,1,4,0,0,6306634,8.966558,42.487621,45.0,12.1,1031.633333,0.014370,38.0,7.016667,70.844595,NaN
2,경남,2016,1,1,4,0,0,1695187,10.962802,29.128224,79.0,17.7,1031.175000,0.029582,39.3,5.616667,52.099421,NaN
3,경북,2016,1,1,4,0,0,1356568,14.384461,38.389385,66.0,20.0,1030.309524,0.033232,41.5,6.233333,57.314554,NaN
4,광주,2016,1,1,4,0,0,729995,9.572120,20.475000,40.0,10.0,1032.200000,0.025175,37.4,5.850000,62.837838,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,전남,2016,31,12,5,0,0,951889,16.677050,33.635000,49.0,13.7,1030.607692,0.025507,42.6,2.573333,65.555094,NaN
6218,전북,2016,31,12,5,0,0,927505,15.011563,52.149758,42.0,14.5,1030.586667,0.021604,41.4,3.466667,72.161486,NaN
6219,제주,2016,31,12,5,0,0,322545,11.278426,28.460474,31.0,8.0,1029.900000,0.039094,39.2,3.046667,60.569257,NaN
6220,충남,2016,31,12,5,0,0,1064765,13.768860,71.473958,41.0,11.7,1030.100000,0.019109,40.7,2.520000,75.118243,NaN


In [21]:
hap_male = pd.concat([test_male2[['area','yyyy','mm','dd']], prediction_male['Label']], axis=1)
hap_male['sex'] = 1
hap_male

,area,yyyy,mm,dd,Label,sex
0,강원,2016,1,1,1.113011,1
1,경기,2016,1,1,4.599620,1
2,경남,2016,1,1,1.975583,1
3,경북,2016,1,1,1.867771,1
4,광주,2016,1,1,0.646251,1
...,...,...,...,...,...,...
6217,전남,2016,12,31,1.190487,1
6218,전북,2016,12,31,0.913326,1
6219,제주,2016,12,31,0.040178,1
6220,충남,2016,12,31,0.875832,1


In [22]:
hap_female = pd.concat([test_female2[['area','yyyy','mm','dd']], prediction_female['Label']], axis=1)
hap_female['sex'] = 2
hap_female

,area,yyyy,mm,dd,Label,sex
0,강원,2016,1,1,0.952964,2
1,경기,2016,1,1,4.388139,2
2,경남,2016,1,1,2.037222,2
3,경북,2016,1,1,1.912801,2
4,광주,2016,1,1,0.777628,2
...,...,...,...,...,...,...
6217,전남,2016,12,31,1.495075,2
6218,전북,2016,12,31,1.266596,2
6219,제주,2016,12,31,0.081403,2
6220,충남,2016,12,31,0.898179,2


In [23]:
verify = pd.read_csv('./edited_data/2-2_검증데이터셋.csv', encoding='cp949')
verify

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,NaN
1,2016-01-01,경기,1,NaN
2,2016-01-01,경남,1,NaN
3,2016-01-01,경북,1,NaN
4,2016-01-01,광주,1,NaN
...,...,...,...,...
12439,2016-12-31,전남,2,NaN
12440,2016-12-31,전북,2,NaN
12441,2016-12-31,제주,2,NaN
12442,2016-12-31,충남,2,NaN


In [24]:
hap_male['yyyy'] = hap_male['yyyy'].astype('int')
hap_male['mm'] = hap_male['mm'].astype('int')
hap_male['dd'] = hap_male['dd'].astype('int')

hap_female['yyyy'] = hap_female['yyyy'].astype('int')
hap_female['mm'] = hap_female['mm'].astype('int')
hap_female['dd'] = hap_female['dd'].astype('int')

In [25]:
hap_male['dateInt']=hap_male['yyyy'].astype(str) + hap_male['mm'].astype(str).str.zfill(2)+ hap_male['dd'].astype(str).str.zfill(2)
hap_male['date'] = pd.to_datetime(hap_male['dateInt'], format='%Y%m%d')
hap_male.drop(columns=['dateInt'], inplace=True)

hap_female['dateInt']=hap_female['yyyy'].astype(str) + hap_female['mm'].astype(str).str.zfill(2)+ hap_female['dd'].astype(str).str.zfill(2)
hap_female['date'] = pd.to_datetime(hap_female['dateInt'], format='%Y%m%d')
hap_female.drop(columns=['dateInt'], inplace=True)

In [26]:
hap_male.sort_values(['date', 'area']).reset_index(drop=True)
hap_female.sort_values(['date', 'area']).reset_index(drop=True)


,area,yyyy,mm,dd,Label,sex,date
0,강원,2016,1,1,0.952964,2,2016-01-01
1,경기,2016,1,1,4.388139,2,2016-01-01
2,경남,2016,1,1,2.037222,2,2016-01-01
3,경북,2016,1,1,1.912801,2,2016-01-01
4,광주,2016,1,1,0.777628,2,2016-01-01
...,...,...,...,...,...,...,...
6217,전남,2016,12,31,1.495075,2,2016-12-31
6218,전북,2016,12,31,1.266596,2,2016-12-31
6219,제주,2016,12,31,0.081403,2,2016-12-31
6220,충남,2016,12,31,0.898179,2,2016-12-31


In [27]:
hap = pd.concat([hap_male, hap_female], axis=0).reset_index(drop=True)
hap

,area,yyyy,mm,dd,Label,sex,date
0,강원,2016,1,1,1.113011,1,2016-01-01
1,경기,2016,1,1,4.599620,1,2016-01-01
2,경남,2016,1,1,1.975583,1,2016-01-01
3,경북,2016,1,1,1.867771,1,2016-01-01
4,광주,2016,1,1,0.646251,1,2016-01-01
...,...,...,...,...,...,...,...
12439,전남,2016,12,31,1.495075,2,2016-12-31
12440,전북,2016,12,31,1.266596,2,2016-12-31
12441,제주,2016,12,31,0.081403,2,2016-12-31
12442,충남,2016,12,31,0.898179,2,2016-12-31


In [28]:
verify['frequency'] = hap['Label']
verify

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,1.113011
1,2016-01-01,경기,1,4.599620
2,2016-01-01,경남,1,1.975583
3,2016-01-01,경북,1,1.867771
4,2016-01-01,광주,1,0.646251
...,...,...,...,...
12439,2016-12-31,전남,2,1.495075
12440,2016-12-31,전북,2,1.266596
12441,2016-12-31,제주,2,0.081403
12442,2016-12-31,충남,2,0.898179


In [29]:
verify[verify['frequency']<0] = 0

In [31]:
verify.to_csv('220023_0802_1.csv', encoding='cp949')